In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
file_path = 'drive/MyDrive/+Renzoe Box/Administrative/Team Resources/Team Folders/Quhan Peng/RenzoeBoxV_sample800_NLP_trainingset_corrected.csv'
data = pd.read_csv(file_path)
data.head(1)

,Unnamed: 0,id,handle,title,body,vendor,type,tags,published,option1 name,option1 value,option2 name,option2 value,option3 name,option3 value,variant sku,variant grams,variant inventory tracker,variant inventory qty,variant inventory policy,variant fulfillment service,variant price,variant compare at price,variant requires shipping,variant taxable,variant barcode,image src,image position,image alt text,gift card,variant image,variant weight unit,cost per item,status,renzoe tags,id.1,unnamed: 0,unnamed: 0.1,pod nonpod,product type,coverage type,finish,skin concern,product consistency,product preference,other
0,1861.0,1462.0,bb-crayon-concealer-touch-up-stick,bb crayon concealer & touch-up stick,NaN,Erborian,bb & cc creams,"['face', 'non-pod']",FALSE,Colors,Caramel (for tan to deep skin),NaN,NaN,NaN,NaN,2569687,2.83495,Shopify,0.0,Continue,manual,26.0,NaN,True,True,NaN,https://images.ulta.com/is/image/Ulta/2569686,4.0,NaN,False,['https://images.ulta.com/is/image/Ulta/256968...,g,26.0,draft,NaN,NaN,13987.0,13987.0,"pod, non-pod",Concealer,Medium Coverage,Semi-Matte,Blemishes,NaN,"Clean Ingredients, Cruelty-Free, Gluten-Free",NaN


In [3]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

#set english stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Apply basic NLP for one piece of text
def clean_text(text):
#remove \n\w\s, all lower case
    text = text.replace('\n', ' ')
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    lst_text = text.split()
    #Remove stopwords
    lst_text = [word for word in lst_text if word not in stop_words]
    #Stemming (remove -ing, -ly, ...)
    ps = nltk.stem.porter.PorterStemmer()
    lst_text = [ps.stem(word) for word in lst_text]
    # Lemmatisation (convert the word into root word)
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    lst_text = [lem.lemmatize(word) for word in lst_text]
    text = " ".join(lst_text)
    
    return text

# Apply clean_text function the 'text' column for each row in df 
def NLPCleaning(df):
    data = df.copy()
    data["text"] =data["text"].apply(clean_text)
    return data

In [5]:
def transferY(theY):
  newY = []
  for i in range(len(theY)):
    new_item = theY.iloc[i].lower().split(', ')
    newY.append(new_item)
  return newY

## Finish

In [6]:
fdata = data[['handle', 'title', 'body','finish']]
fdata = fdata.replace('-', np.nan)
fdata.dropna(inplace = True)
fdata = fdata.drop_duplicates()

fdata = fdata[fdata["finish"].str.contains("Metallic")==False]
fdata['text'] = fdata['handle'] + " " + fdata['title'] + " " + fdata['body']

fdata = NLPCleaning(fdata)

X_f = fdata['text']
fdata_y = fdata['finish']

In [7]:
for i in range(len(fdata_y)):

  # fdata_y.values[i] = fdata_y.values[i].strip(" ")

  #replace all the typos
  fdata_y.values[i] = fdata_y.values[i].replace('Foundation', 'foundation')
  fdata_y.values[i] = fdata_y.values[i].replace('Eyeshadow', 'eyeshadow')
  fdata_y.values[i] = fdata_y.values[i].replace('Concealer', 'concealer')
  fdata_y.values[i] = fdata_y.values[i].replace('Primer', 'primer')
  fdata_y.values[i] = fdata_y.values[i].replace('Setting Powder', 'setting powder')
  fdata_y.values[i] = fdata_y.values[i].replace('Eyeliner', 'eyeliner')
  fdata_y.values[i] = fdata_y.values[i].replace('Brow Pencil/Powder/Gel', 'brow')
  fdata_y.values[i] = fdata_y.values[i].replace('Brow Powder', 'brow')
  fdata_y.values[i] = fdata_y.values[i].replace('lip gloss', 'lip')
  fdata_y.values[i] = fdata_y.values[i].replace('lip liner', 'lip')
  fdata_y.values[i] = fdata_y.values[i].replace('lip stick', 'lip')
  fdata_y.values[i] = fdata_y.values[i].replace('lipstick', 'lip')
  fdata_y.values[i] = fdata_y.values[i].replace('Lip Tint/Balm', 'lip')
  fdata_y.values[i] = fdata_y.values[i].replace('Natural', 'natural')
  fdata_y.values[i] = fdata_y.values[i].replace('Natural ', 'natural')
  fdata_y.values[i] = fdata_y.values[i].replace('natural ', 'natural')
  fdata_y.values[i] = fdata_y.values[i].replace('Shimmer', 'shimmer')
  fdata_y.values[i] = fdata_y.values[i].replace('Matte', 'matte')
  fdata_y.values[i] = fdata_y.values[i].replace('Semi-Matte', 'semi-matte')
  fdata_y.values[i] = fdata_y.values[i].replace('Sheer', 'sheer')
  fdata_y.values[i] = fdata_y.values[i].replace('Metallic', 'metallic')
  fdata_y.values[i] = fdata_y.values[i].replace('Mate', 'matte')
  fdata_y.values[i] = fdata_y.values[i].replace('matte ', 'matte')
  fdata_y.values[i] = fdata_y.values[i].replace('Natrual', 'natural')
  fdata_y.values[i] = fdata_y.values[i].replace('Semi-matte', 'semi-matte')
  fdata_y.values[i] = fdata_y.values[i].replace('Dewy', 'dewy')
  fdata_y.values[i] = fdata_y.values[i].replace('Glossy', 'glossy')

## Product Preference

In [8]:
ppdata = data[['handle', 'title', 'body', 'product preference']]

# Transfer every '-' to be NAN
ppdata = ppdata.replace('-', np.nan)
ppdata.dropna(axis = 0, inplace = True)
ppdata.drop_duplicates(inplace = True, ignore_index = True)

ppdata['text'] = ppdata['handle'] + " " + ppdata['title'] + " " + ppdata['body']

ppdata = NLPCleaning(ppdata)
ppdata_y = transferY(ppdata['product preference'])

In [9]:
# Using the resluts from the previous step, we need to manually

# Correct all the typos and formate for y label
for i in range(len(ppdata_y)):
  for j in range(len(ppdata_y[i])):

    #delete all the accident blank after words
    ppdata_y[i][j] = ppdata_y[i][j].strip(" ")

    # Replace all the typos!!!
    # clean ingredients
    # Cruelty Free
    ppdata_y[i][j] = ppdata_y[i][j].replace('cruelty-free,long lasting', 'cruelty free')
    ppdata_y[i][j] = ppdata_y[i][j].replace('cruelty-free', 'cruelty free')
    ppdata_y[i][j] = ppdata_y[i][j].replace('cruelty-free. lengthening', 'cruelty free')
    # Fragrance-Free
    ppdata_y[i][j] = ppdata_y[i][j].replace('fragrance free', 'fragrance-free')
    ppdata_y[i][j] = ppdata_y[i][j].replace('fragrance- free', 'fragrance-free')
    ppdata_y[i][j] = ppdata_y[i][j].replace('fragrance-free,oil free', 'fragrance-free')
    ppdata_y[i][j] = ppdata_y[i][j].replace('fragrnace free', 'fragrance-free')
    ppdata_y[i][j] = ppdata_y[i][j].replace('fragrange free', 'fragrance-free')
    # USA Made, Vegan, Minority Founded, SPF, Ethically Sourced, Trending, Protection
    # Woman Founded
    ppdata_y[i][j] = ppdata_y[i][j].replace('women founded', 'woman founded')
    # Long Lasting
    ppdata_y[i][j] = ppdata_y[i][j].replace('long  lasting', 'long lasting')
    ppdata_y[i][j] = ppdata_y[i][j].replace('long wearing', 'long lasting')
    ppdata_y[i][j] = ppdata_y[i][j].replace('long-lasting', 'long lasting')
    ppdata_y[i][j] = ppdata_y[i][j].replace('longlasting', 'long lasting')

ppdata.dropna(axis = 0, inplace = True)

product_preference_cols = ['clean ingredients', 'cruelty free', 'fragrance-free', 'usa made', 
                           'vegan', 'minority founded', 'spf', 'ethically sourced', 'trending', 
                           'protection', 'woman founded', 'long lasting']

valid_index = set()
new_ppdata_y = []

for i in range(len(ppdata_y)):
  if all([item in product_preference_cols for item in ppdata_y[i]]):
    # print(ppdata_y[i])
    new_ppdata_y.append(ppdata_y[i])
    valid_index.add(i)

X_pp = ppdata.loc[valid_index, 'text']

## Skin Concern

In [10]:
scdata = data[['handle', 'title', 'body','skin concern']]

# transfer every '-' to be NAN
scdata = scdata.replace('-', np.nan)
scdata = scdata.dropna()
scdata = scdata.drop_duplicates()

scdata['text'] = scdata['handle'] + " " + scdata['title'] + " " + scdata['body']
scdata = NLPCleaning(scdata)

X_sc = scdata['text']
scdata_y = transferY(scdata['skin concern'])

In [11]:
# correct all the typos and formate for y label
for i in range(len(scdata_y)):
  for j in range(len(scdata_y[i])):
    #delete all the accident blank after words
    scdata_y[i][j] = scdata_y[i][j].rstrip(" ")
    #replace all the typos
    scdata_y[i][j] = scdata_y[i][j].replace('blemishes', 'blemish')
    scdata_y[i][j] = scdata_y[i][j].replace('combiation', 'combination')
    scdata_y[i][j] = scdata_y[i][j].replace('combinatoin', 'combination')
    scdata_y[i][j] = scdata_y[i][j].replace('comination', 'combination')
    scdata_y[i][j] = scdata_y[i][j].replace('lines lines', 'fine lines')
    scdata_y[i][j] = scdata_y[i][j].replace('hyper pigmentation', 'hyperpigmentation')
    scdata_y[i][j] = scdata_y[i][j].replace('hyper pigmentatoins', 'hyperpigmentation')
    scdata_y[i][j] = scdata_y[i][j].replace('hypermpigmentaiton', 'hyperpigmentation')
    scdata_y[i][j] = scdata_y[i][j].replace('hyperpigmenation', 'hyperpigmentation')
    scdata_y[i][j] = scdata_y[i][j].replace('hyperpigmentaiton', 'hyperpigmentation')
    scdata_y[i][j] = scdata_y[i][j].replace('hyperpigmentations', 'hyperpigmentation')
    scdata_y[i][j] = scdata_y[i][j].replace('oliy', 'oily')
    scdata_y[i][j] = scdata_y[i][j].replace('large pores', 'pores')
    scdata_y[i][j] = scdata_y[i][j].replace('pores', 'large pores')
    scdata_y[i][j] = scdata_y[i][j].replace('oil free', 'blemish')

## Coverage Type

In [12]:
ctdata = data[['handle', 'title', 'body', 'coverage type']]

# Transfer every '-' to be NAN
ctdata = ctdata.replace('-', np.nan)
ctdata.dropna(axis = 0, inplace = True)
ctdata.drop_duplicates(inplace = True, ignore_index = True)

ctdata['text'] = ctdata['handle'] + " " + ctdata['title'] + " " + ctdata['body']

ctdata = NLPCleaning(ctdata)

X_ct = ctdata['text']
ctdata_y = ctdata['coverage type']

In [13]:
# correct all the typos and formate for y label

for i in range(len(ctdata_y)):

  if(i >= len(ctdata_y)): break

  if(ctdata_y.values[i] == ' ' or ctdata_y.values[i] == 'metallic'):
    ctdata_y = ctdata_y.drop(ctdata_y.index[i])
    X_ct.drop(X_ct.index[i], inplace = True)
    continue

  if(ctdata_y.values[i] == 'full'):
    ctdata_y.values[i] = 'full coverage'
    continue
  
  if(ctdata_y.values[i] == 'medium' or ctdata_y.values[i] == 'Medium'):
    ctdata_y.values[i] = 'medium coverage'
    continue

for i in range(len(ctdata_y)):
    
    #delete all the accident blank after words
    ctdata_y.values[i] = ctdata_y.values[i].rstrip(" ")
    #replace all the typos
    ctdata_y.values[i] = ctdata_y.values[i].replace('Full Coverage', 'full coverage')
    ctdata_y.values[i] = ctdata_y.values[i].replace('Full coverage', 'full coverage')
    ctdata_y.values[i] = ctdata_y.values[i].replace('Medium Coverage', 'medium coverage')
    ctdata_y.values[i] = ctdata_y.values[i].replace('Light Coverage', 'light coverage')

In [14]:
print(X_ct.shape)
print(ctdata_y.shape)

(152,)
(152,)


## Product Type

In [15]:
ptdata = data[['handle', 'title', 'body','product type']]

# transfer every '-' to be NAN
ptdata = ptdata.replace('-', np.nan)
ptdata = ptdata.dropna()
ptdata = ptdata.drop_duplicates()

ptdata['text'] = ptdata['handle'] + " " + ptdata['title'] + " " + ptdata['body']
ptdata = NLPCleaning(ptdata)

ptdata = ptdata[~ptdata['product type'].isin(['duplicate?'])]  
X_pt = ptdata['text']
ptdata_y = ptdata['product type']

In [16]:
for i in range(len(ptdata_y)):

  ptdata_y.values[i] = ptdata_y.values[i].lstrip(" ")

  #replace all the typos
  ptdata_y.values[i] = ptdata_y.values[i].replace('Foundation', 'foundation')
  ptdata_y.values[i] = ptdata_y.values[i].replace('Eyeshadow', 'eyeshadow')
  ptdata_y.values[i] = ptdata_y.values[i].replace('Concealer', 'concealer')
  ptdata_y.values[i] = ptdata_y.values[i].replace('Primer', 'primer')
  ptdata_y.values[i] = ptdata_y.values[i].replace('Setting Powder', 'setting powder')
  ptdata_y.values[i] = ptdata_y.values[i].replace('Eyeliner', 'eyeliner')
  ptdata_y.values[i] = ptdata_y.values[i].replace('Brow Pencil/Powder/Gel', 'brow')
  ptdata_y.values[i] = ptdata_y.values[i].replace('Brow Powder', 'brow')
  ptdata_y.values[i] = ptdata_y.values[i].replace('lip gloss', 'lip')
  ptdata_y.values[i] = ptdata_y.values[i].replace('lip liner', 'lip')
  ptdata_y.values[i] = ptdata_y.values[i].replace('lip stick', 'lip')
  ptdata_y.values[i] = ptdata_y.values[i].replace('lipstick', 'lip')
  ptdata_y.values[i] = ptdata_y.values[i].replace('Lip Tint/Balm', 'lip')

## Product Consistency

In [17]:
file_path = 'drive/MyDrive/+Renzoe Box/Administrative/Team Resources/Team Folders/+2021 Fall Cohort - Group Projects/Taggers/RenzoeBoxV_sample800_NLP_sprint_trainingset.csv'
pcdataset = pd.read_csv(file_path)
pcdataset.head(3)

,id,handle,title,body,vendor,type,tags,image src,pod nonpod,product consistency
0,5301,skin-glazing,skin glazing,\ndescription\nnabla skin glazing is a kaleido...,NABLA,highlighter,"['face', 'pod']",https://images.ulta.com/is/image/Ulta/2558304,pod,pressed powder
1,1462,infallible-fresh-wear-24hr-foundation,infallible fresh wear 24hr foundation,\ndescription\nbe unstoppable. introducing l'o...,L'Or??al,foundation,"['face', 'non-pod']",https://images.ulta.com/is/image/Ulta/2538150,non-pod,"liquid, bottle"
2,14007,sos-primer,sos primer,\ndescription\nprime time! discover solutions ...,Clarins,face primer,"['face', 'non-pod']",https://images.ulta.com/is/image/Ulta/2523525,non-pod,liquid


In [18]:
pcdata = pcdataset[['handle', 'title', 'body', 'product consistency']]

# transfer every '-' to be NAN
pcdata = pcdata.replace('-', np.nan)
pcdata = pcdata.dropna()
pcdata = pcdata.drop_duplicates()

pcdata['text'] = pcdata['handle'] + " " + pcdata['title'] + " " + pcdata['body']
pcdata = NLPCleaning(pcdata)
pcdata_y = pcdata['product consistency']

In [19]:
for i in range(len(pcdata)):

  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].lstrip(" ")

  #replace all the typos
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('liquid, tube', 'liquid')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('liquid, bottle', 'liquid')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('solid stick', 'stick')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('cream, tube', 'cream')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('tube', 'pencil')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('pressed powder, palette', 'pressed powder')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('cream, liquid', 'liquid')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('pencil, tube', 'pencil')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('cream, cake', 'cream')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('liquid, pencil', 'pencil')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('gel', 'liquid')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('cream, pencil', 'pencil')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('balm', 'cream')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('cream, solid stick', 'stick')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('eyelashes', 'TOOLS')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('cream to powder', 'cream')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('gel, tube', 'liquid')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('solid stick, pencil', 'stick')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('tube, pencil', 'pencil')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('PHOTO DOESNT MATCH', "PHOTO DOESN'T MATCH")
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('SKINCARE', 'TOOLS')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('pencil, pencil', 'pencil')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('liquid, pencil', 'liquid')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('stick, pencil', 'pencil')
  pcdata['product consistency'].values[i] = pcdata['product consistency'].values[i].replace('cream, stick', 'cream')

pcdata = pcdata[~(pcdata['product consistency'] == "PHOTO DOESN'T MATCH" )]
pcdata = pcdata[~(pcdata['product consistency'] == "loose powder" )]

X_pc = pcdata['text']
pcdata_y = pcdata['product consistency']
print(X_pc.shape)
print(pcdata_y.shape)

(541,)
(541,)


### Modeling

In [20]:
import pickle
from sklearn.model_selection import train_test_split
# Finish
X_train_f, X_test_f, f_y_train, f_y_test = train_test_split(X_f, fdata_y, test_size=0.33)
f_model_name = 'finish_finalized_model.sav'
f_vec = 'finish_vectorizer.sav'

f_loaded_model = pickle.load(open(f_model_name, 'rb'))
f_loaded_vec = pickle.load(open(f_vec, 'rb'))

X_test_f = f_loaded_vec.transform(X_test_f)
f_y_pred = f_loaded_model.predict(X_test_f)

In [21]:
# Coverage Type
X_train_ct, X_test_ct, ct_y_train, ct_y_test = train_test_split(X_ct, ctdata_y, test_size=0.33)

ct_model_name = 'coverage_type_finalized_model.sav'
ct_vec = 'coverage_type_vectorizer.sav'

ct_loaded_model = pickle.load(open(ct_model_name, 'rb'))
ct_loaded_vec = pickle.load(open(ct_vec, 'rb'))

X_test_ct = ct_loaded_vec.transform(X_test_ct)
ct_y_pred = ct_loaded_model.predict(X_test_ct)

In [22]:
## Product Type
X_train_pt, X_test_pt, pt_y_train, pt_y_test = train_test_split(X_pt, ptdata_y, test_size=0.33)

pt_model_name = 'product_type_finalized_model.sav'
pt_vec = 'product_type_vectorizer.sav'

pt_loaded_model = pickle.load(open(pt_model_name, 'rb'))
pt_loaded_vec = pickle.load(open(pt_vec, 'rb'))

X_test_pt = pt_loaded_vec.transform(X_test_pt)
pt_y_pred = pt_loaded_model.predict(X_test_pt)

In [23]:
## Product Consistency
X_train_pc, X_test_pc, pc_y_train, pc_y_test = train_test_split(X_pc, pcdata_y, test_size = 0.33)

pc_model_name = 'product_consistency_finalized_model.sav'
pc_vec = 'product_consistency_vectorizer.sav'

pc_loaded_model = pickle.load(open(pc_model_name, 'rb'))
pc_loaded_vec = pickle.load(open(pc_vec, 'rb'))

X_test_pc = pc_loaded_vec.transform(X_test_pc)
pc_y_pred = pc_loaded_model.predict(X_test_pc)

In [29]:
# Product Preference
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

X_train_pp, X_test_pp, pp_y_train, pp_y_test = train_test_split(X_pp, new_ppdata_y, test_size = 0.33)

pp_model_name = 'product_preference_finalized_model.sav'
pp_vec = 'product_preference_vectorizer.sav'
pp_mlb = 'product_preference_mlb.sav'

pp_loaded_model = pickle.load(open(pp_model_name, 'rb'))
pp_loaded_vec = pickle.load(open(pp_vec, 'rb'))
pp_loaded_mlb = pickle.load(open(pp_mlb, 'rb'))

pp_train_labels = pp_loaded_mlb.fit_transform(pp_y_train)
pp_test_labels = pp_loaded_mlb.transform(pp_y_test)

X_test_pp = pp_loaded_vec.transform(X_test_pp)
pp_y_pred = pp_loaded_model.predict(X_test_pp)
pp_pred_labels = pp_loaded_mlb.transform(pp_y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['usa made'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) [0, 1] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))


In [30]:
# Skin Concern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
X_train_sc, X_test_sc, sc_y_train, sc_y_test = train_test_split(X_sc, scdata_y, test_size=0.33)

sc_model_name = 'skin_concern_finalized_model.sav'
sc_vec = 'skin_concern_vectorizer.sav'
sc_mlb = 'skin_concern_mlb.sav'

sc_loaded_model = pickle.load(open(sc_model_name, 'rb'))
sc_loaded_vec = pickle.load(open(sc_vec, 'rb'))
sc_loaded_mlb = pickle.load(open(sc_mlb, 'rb'))

sc_train_labels = sc_loaded_mlb.fit_transform(sc_y_train)
sc_test_labels = sc_loaded_mlb.transform(sc_y_test)

X_test_sc = sc_loaded_vec.transform(X_test_sc)
sc_y_pred = sc_loaded_model.predict(X_test_sc)
sc_pred_labels = mlb.transform(sc_y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) [0, 1] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))


## Evaluation

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss
ModelsPerformance = {}

def metricsReport_new(modelName, test_labels, predictions):

    macro_f1 = f1_score(test_labels, predictions, average='macro')
    micro_f1 = f1_score(test_labels, predictions, average='micro')
    
    hamLoss = hamming_loss(test_labels, predictions)
    ModelsPerformance[modelName] = micro_f1
    max_score = max(micro_f1, macro_f1, hamLoss)
    
    return modelName, max_score

In [32]:
# Unique label
f_scores = metricsReport_new(f_loaded_model, f_y_test, f_y_pred)
ct_scores = metricsReport_new(ct_loaded_model, ct_y_test, ct_y_pred)
pt_scores = metricsReport_new(pt_loaded_model, pt_y_test, pt_y_pred)
pc_scores = metricsReport_new(pc_loaded_model, pc_y_test, pc_y_pred)

# Multi-label
pp_scores = metricsReport_new(pp_loaded_model, pp_test_labels, pp_pred_labels)
sc_scores = metricsReport_new(sc_loaded_model, sc_test_labels, sc_pred_labels)

results = pd.DataFrame([f_scores, ct_scores, pt_scores, pc_scores, pp_scores, sc_scores], 
                       columns = ["model", "score"], 
                       index = ['Finish', 'Coverage Type', 'Product Type', 
                              'Product Consistency', 'Product Preference', 'Skin Concern'])
results

,model,score
Finish,"MLPClassifier(alpha=0.01, hidden_layer_sizes=(...",0.871560
Coverage Type,"SVC(C=10, kernel='sigmoid', random_state=1)",0.901961
Product Type,"SVC(C=10, kernel='sigmoid', random_state=1)",0.988816
Product Consistency,SVC(),0.849162
Product Preference,DecisionTreeClassifier(),0.245690
Skin Concern,"OneVsRestClassifier(estimator=LinearSVC(), n_j...",0.273224
